In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Load local TAZ shapefile
taz_gdf = gpd.read_file(r"data\USTMv4_20250729\USTMv4_20250729.shp")

# Step 2: Load lookup layer from ArcGIS FeatureServer
lookup_url = (
    "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/"
    "TAZ_GeographyLookup_082025/FeatureServer/0/query"
    "?where=1=1&outFields=*&f=geojson"
)
lookup_gdf = gpd.read_file(lookup_url)

# Step 3: Force both layers to EPSG:26912 (UTM Zone 12N)
target_crs = "EPSG:26912"
taz_gdf = taz_gdf.to_crs(target_crs)
lookup_gdf = lookup_gdf.to_crs(target_crs)

# Step 4: Compute centroids of TAZ polygons
taz_gdf["centroid"] = taz_gdf.geometry.centroid
taz_centroids = taz_gdf.set_geometry("centroid")

# Step 5: Spatial join — find which lookup polygons contain each TAZ centroid
joined = gpd.sjoin(
    taz_centroids,
    lookup_gdf[["GEONAME", "GEOTYPE", "geometry"]],
    how="inner",
    predicate="within"
)

# Step 6: Merge the GEONAME and GEOTYPE fields back into the original TAZ polygons
taz_with_geo = taz_gdf.merge(
    joined[["GEONAME", "GEOTYPE"]],
    left_index=True,
    right_index=True,
    how="inner"
)

# Optional: clean up
taz_with_geo = taz_with_geo.drop(columns="centroid")


In [2]:
taz_with_geo

,CO_IDX,CO_TAZID,SUBAREAID,ACRES,DEVACRES,DEVPBLEPCT,X,Y,geometry,GEONAME,GEOTYPE
121,1.0,3001.0,0.0,1907.010233,1849.799926,0.970000,397068.618562,4.647978e+06,"POLYGON ((397636.173 4649161.877, 397659.666 4...",Box Elder County,COUNTYNAME
122,2.0,3002.0,0.0,784.943360,745.696192,0.950000,396117.723243,4.649841e+06,"POLYGON ((395440.72 4650675.05, 395839.76 4650...",Box Elder County,COUNTYNAME
122,2.0,3002.0,0.0,784.943360,745.696192,0.950000,396117.723243,4.649841e+06,"POLYGON ((395440.72 4650675.05, 395839.76 4650...",Unincorporated Box Elder County,CITYNAME
123,3.0,3003.0,0.0,1333.039635,1306.378842,0.980000,398781.357195,4.649222e+06,"POLYGON ((399949.828 4648868.891, 400066.253 4...",Box Elder County,COUNTYNAME
123,3.0,3003.0,0.0,1333.039635,1306.378842,0.980000,398781.357195,4.649222e+06,"POLYGON ((399949.828 4648868.891, 400066.253 4...",Unincorporated Box Elder County,CITYNAME
...,...,...,...,...,...,...,...,...,...,...,...
9889,157.0,490157.0,1.0,280.469204,263.337797,0.938919,417306.117258,4.465355e+06,"POLYGON ((417743.195 4465657.155, 416190.446 4...",Wasatch Front,LARGEAREA
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Utah County,COUNTYNAME
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Orem,CITYNAME
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",MAG,MPO


In [3]:
taz_with_geo.groupby(['CO_TAZID'], as_index=False).agg(GEONAME=('GEONAME', 'count'))

,CO_TAZID,GEONAME
0,3001.0,1
1,3002.0,2
2,3003.0,2
3,3004.0,2
4,3005.0,2
...,...,...
4978,570424.0,2
4979,570425.0,2
4980,570426.0,2
4981,570427.0,2


In [4]:
taz_with_geo[taz_with_geo['CO_TAZID'] == 490719]  # Example filter for a specific TAZ

,CO_IDX,CO_TAZID,SUBAREAID,ACRES,DEVACRES,DEVPBLEPCT,X,Y,geometry,GEONAME,GEOTYPE
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.08164,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Utah County,COUNTYNAME
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.08164,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Orem,CITYNAME
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.08164,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",MAG,MPO
9890,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.08164,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Wasatch Front,LARGEAREA


In [5]:
# Step 2: Load ATO data for both years
ato_2019 = pd.read_csv(r"data\ATO\WFv911_BY_2019_Access_to_Opportunity.csv")
ato_2023 = pd.read_csv(r"data\ATO\WFv911_OY_2023_Access_to_Opportunity.csv")

# Step 3: Add 'YEAR' column to each
ato_2019["YEAR"] = 2019
ato_2023["YEAR"] = 2023

# Step 4: Combine the two ATO datasets
ato_df = pd.concat([ato_2019, ato_2023], ignore_index=True)

# Step 5: Inspect key column for join
print("Columns in ATO:", ato_df.columns)
print("Columns in TAZ:", taz_gdf.columns)

# Assume both have a TAZ identifier column like 'CO_TAZID'
# Adjust this if needed — example uses 'CO_TAZID'
common_key = "CO_TAZID"  # or replace with actual key column like 'TAZ'

# Join on common TAZ ID field
# Replace 'CO_TAZID' with your actual ID column name if different
taz_with_geo_ato = taz_with_geo.merge(
    ato_df,
    how="inner",
    left_on="CO_TAZID",   # adjust this if your TAZ ID column is named differently
    right_on="CO_TAZID"
)

# Step 7: Confirm result
taz_with_geo_ato


Columns in ATO: Index(['TAZID', 'CO_TAZID', 'DevAcres', 'HH', 'Job', 'Job_byAuto',
       'Job_byTran', 'Job_byBike', 'Job_byWalk', 'act_Job_byAuto',
       'act_Job_byTran', 'act_Job_byBike', 'act_Job_byWalk', 'Loss_Job_Cong',
       'Loss_Job_Net', 'Loss_Job_Abs', 'act_Loss_Job_Cong', 'act_Loss_Job_Net',
       'act_Loss_Job_Abs', 'HH_byAuto', 'HH_byTran', 'HH_byBike', 'HH_byWalk',
       'act_HH_byAuto', 'act_HH_byTran', 'act_HH_byBike', 'act_HH_byWalk',
       'Loss_HH_Cong', 'Loss_HH_Net', 'Loss_HH_Abs', 'act_Loss_HH_Cong',
       'act_Loss_HH_Net', 'act_Loss_HH_Abs', 'tmp_Job_byTran_wk',
       'tmp_Job_byTran_dr', 'tmp_HH_byTran_wk', 'tmp_HH_byTran_dr',
       'tmp_Job_byAuto_FF', 'tmp_Job_byAuto_SL', 'tmp_HH_byAuto_FF',
       'tmp_HH_byAuto_SL', 'YEAR'],
      dtype='object')
Columns in TAZ: Index(['CO_IDX', 'CO_TAZID', 'SUBAREAID', 'ACRES', 'DEVACRES', 'DEVPBLEPCT',
       'X', 'Y', 'geometry', 'centroid'],
      dtype='object')


,CO_IDX,CO_TAZID,SUBAREAID,ACRES,DEVACRES,DEVPBLEPCT,X,Y,geometry,GEONAME,...,act_Loss_HH_Abs,tmp_Job_byTran_wk,tmp_Job_byTran_dr,tmp_HH_byTran_wk,tmp_HH_byTran_dr,tmp_Job_byAuto_FF,tmp_Job_byAuto_SL,tmp_HH_byAuto_FF,tmp_HH_byAuto_SL,YEAR
0,1.0,30001.0,1.0,582.087210,581.205980,0.998486,412339.399850,4.603960e+06,"POLYGON ((412861.8 4605425.5, 412857.6 4605424...",Box Elder County,...,0.0,0,0,0,0,33874,83386,20936,53830,2019
1,1.0,30001.0,1.0,582.087210,581.205980,0.998486,412339.399850,4.603960e+06,"POLYGON ((412861.8 4605425.5, 412857.6 4605424...",Box Elder County,...,0.0,0,0,0,0,36206,89381,22506,57756,2023
2,1.0,30001.0,1.0,582.087210,581.205980,0.998486,412339.399850,4.603960e+06,"POLYGON ((412861.8 4605425.5, 412857.6 4605424...",Unincorporated Box Elder County,...,0.0,0,0,0,0,33874,83386,20936,53830,2019
3,1.0,30001.0,1.0,582.087210,581.205980,0.998486,412339.399850,4.603960e+06,"POLYGON ((412861.8 4605425.5, 412857.6 4605424...",Unincorporated Box Elder County,...,0.0,0,0,0,0,36206,89381,22506,57756,2023
4,2.0,30002.0,1.0,135.465170,134.408425,0.992199,412640.145346,4.606014e+06,"POLYGON ((413060.9 4605180.7, 413021.2 4605183...",Box Elder County,...,0.0,0,0,0,0,36800,83557,22737,53896,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31705,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Orem,...,-120069788.0,97633,21777,35606,9859,323759,487497,155741,260816,2023
31706,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",MAG,...,-86393675.0,89422,21283,29870,8477,293405,441617,139260,234855,2019
31707,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",MAG,...,-120069788.0,97633,21777,35606,9859,323759,487497,155741,260816,2023
31708,719.0,490719.0,1.0,74.754976,66.192217,0.885456,438235.081640,4.457559e+06,"POLYGON ((438715.777 4457648.867, 438816.633 4...",Wasatch Front,...,-86393675.0,89422,21283,29870,8477,293405,441617,139260,234855,2019


In [6]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Step 1: Define weighted average helper
def weighted_avg(df, value_col, weight_col):
    valid = df[weight_col] > 0
    weighted_sum = (df.loc[valid, value_col] * df.loc[valid, weight_col]).sum()
    total_weight = df.loc[valid, weight_col].sum()
    return weighted_sum / total_weight if total_weight != 0 else np.nan

# Step 2: Define columns to aggregate
job_cols = ['Job_byAuto', 'Job_byTran', 'Job_byBike', 'Job_byWalk']
hh_cols  = ['HH_byAuto', 'HH_byTran', 'HH_byBike', 'HH_byWalk']

# Step 3: Group by GEONAME, GEOTYPE, and YEAR
grouped = []

for (geo, geotype, year), group in taz_with_geo_ato.groupby(['GEONAME', 'GEOTYPE', 'YEAR']):
    row = {
        'GEONAME': geo,
        'GEOTYPE': geotype,
        'YEAR': year
    }

    # Weighted by HH
    for col in job_cols:
        row[col] = weighted_avg(group, value_col=col, weight_col='HH')
    
    # Weighted by Job
    for col in hh_cols:
        row[col] = weighted_avg(group, value_col=col, weight_col='Job')

    grouped.append(row)


# Step 4: Convert to DataFrame
dissolved_df = pd.DataFrame(grouped)

# List of columns to round
cols_to_round = [
    'Job_byAuto', 'Job_byTran', 'Job_byBike',
    'HH_byAuto', 'HH_byTran', 'HH_byBike',
]
cols_to_round_10 = [
    'Job_byWalk',
    'HH_byWalk'
]

# Round all selected columns to the nearest 100
dissolved_df[cols_to_round] = dissolved_df[cols_to_round].round(-2).astype("Int64").fillna(0)

# Round all selected columns to the nearest 100
dissolved_df[cols_to_round_10] = dissolved_df[cols_to_round_10].round(-1).astype("Int64").fillna(0)

# Define renaming map
rename_map = {
    # Job access weighted by HH
    'Job_byAuto': 'HHToJobByAuto',
    'Job_byTran': 'HHToJobByTran',
    'Job_byBike': 'HHToJobByBike',
    'Job_byWalk': 'HHToJobByWalk',

    # HH access weighted by Job
    'HH_byAuto': 'JobtoHHByAuto',
    'HH_byTran': 'JobtoHHByTran',
    'HH_byBike': 'JobtoHHByBike',
    'HH_byWalk': 'JobtoHHByWalk'
}

dissolved_df = dissolved_df.rename(columns=rename_map)

# Step 1: Identify value columns (exclude grouping vars)
value_cols = [col for col in dissolved_df.columns if col not in ['GEONAME', 'GEOTYPE', 'YEAR']]

# Step 2: Pivot table
dissolved_wide_df = dissolved_df.pivot_table(
    index=['GEONAME', 'GEOTYPE'],
    columns='YEAR',
    values=value_cols
)

# Step 3: Flatten MultiIndex columns to single level with suffix
dissolved_wide_df.columns = [f"{col}_{year}" for col, year in dissolved_wide_df.columns]

# Step 4: Reset index to make it a clean DataFrame
dissolved_wide_df = dissolved_wide_df.reset_index()

# only export jobs by auto
dissolved_wide_df = dissolved_wide_df[['GEONAME','GEOTYPE','HHToJobByAuto_2019','HHToJobByAuto_2023']]

# Apply rename
dissolved_wide_df

,GEONAME,GEOTYPE,HHToJobByAuto_2019,HHToJobByAuto_2023
0,Alpine,CITYNAME,214500.0,234000.0
1,Alta,CITYNAME,127800.0,130500.0
2,American Fork,CITYNAME,198600.0,208200.0
3,Bluffdale,CITYNAME,252900.0,275200.0
4,Bountiful,CITYNAME,341100.0,349700.0
...,...,...,...,...
94,West Valley City,CITYNAME,522900.0,581400.0
95,White City,CITYNAME,393000.0,441400.0
96,Willard,CITYNAME,113700.0,123500.0
97,Woodland Hills,CITYNAME,93400.0,91800.0


In [7]:
# Repeat each geometry for each year by merging
output_gdf = dissolved_wide_df.merge(
    lookup_gdf.to_crs("EPSG:4326"),
    on=["GEONAME", "GEOTYPE"],
    how="left"
)

# Convert to GeoDataFrame
output_gdf = gpd.GeoDataFrame(output_gdf, geometry="geometry")

output_gdf = output_gdf.drop(columns=['OBJECTID','GEOSORT','Shape__Area','Shape__Length'])

output_gdf.to_file("output/BIG5_AccessToJobs_Metric.geojson", driver="GeoJSON")